In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df=pd.read_pickle("dataset/cic_processed.pkl")
print(df.head())

        Protocol  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
453520         6           1207             5             2              935   
339493         6           1151             5             2              935   
739247         6           1348             2             0                0   
608461         6           1302             5             2              935   
881208         6          15181             5             2              935   

        TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
453520              314              935                0             187.0   
339493              287              935                0             187.0   
739247                0                0                0               0.0   
608461              325              935                0             187.0   
881208              309              935                0             187.0   

        Fwd Pkt Len Std  ...  Fwd Seg Size M

In [3]:
X=df.drop(columns=['Label'])
Y=df['Label']
original_features=X.columns
print(X.shape,Y.shape,len(X),len(Y))

(425775, 77) (425775,) 425775 425775


In [4]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

In [5]:
from sklearn.feature_selection import SelectKBest, f_classif #feature selection

selector=SelectKBest(score_func=f_classif,k=20)
X_train=selector.fit_transform(X_train,Y_train)
X_test=selector.transform(X_test)
selected_cols=original_features[selector.get_support()]
print(selected_cols)

Index(['Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Mean', 'Flow Byts/s', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Bwd Pkts',
       'Init Fwd Win Byts', 'Init Bwd Win Byts'],
      dtype='object')


C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [31 32 33 49 55 56 57 58 59 60] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [6]:
from sklearn.linear_model import  LogisticRegression
model=LogisticRegression(max_iter=3000)
model.fit(X_train,Y_train)

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=3000)

In [11]:
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score
Y_pred=model.predict(X_test)
train_acc = accuracy_score(Y_train, model.predict(X_train))
test_acc  = accuracy_score(Y_test, model.predict(X_test))
cm=confusion_matrix(Y_test,Y_pred)
print("train accuracy:",train_acc)
print("test accuracy",test_acc)
print("confusion matrix\n",cm)

train accuracy: 0.9999442193646879
test accuracy 0.9999765134167107
confusion matrix
 [[72163     2]
 [    0 12990]]


In [12]:
tn,fp,fn,tp=cm.ravel()
far=fp/(fp+tn)
detect_rate=tp/(tp+fn)
print("False alarm rate:",far)
print("detection rate",detect_rate)

False alarm rate: 2.7714265918381488e-05
detection rate 1.0
